In [ ]:
#
# Attempts to implement the following:
#
# H. Rave, V. Molchanov and L. Linsen, "Uniform Sample Distribution in Scatterplots via Sector-based Transformation," 
# 2024 IEEE Visualization and Visual Analytics (VIS), St. Pete Beach, FL, USA, 2024, pp. 156-160, 
# doi: 10.1109/VIS55277.2024.00039. 
# keywords: {Data analysis;Visual analytics;Clutter;Scatterplot de-cluttering;spatial transformation},
#

In [ ]:
import polars as pl
import numpy as np
from math import cos, sin, pi, sqrt, atan2
from shapely import Polygon
import rtsvg
rt = rtsvg.RACETrack()
df = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk1.csv')
df = rt.columnsAreTimestamps(df, 'parsedDate')
df = df.rename({'TimeSeconds':                '_del1_', 'parsedDate':                 'timestamp',
                'dateTimeStr':                '_del2_', 'ipLayerProtocol':            'pro',
                'ipLayerProtocolCode':        '_del3_', 'firstSeenSrcIp':             'sip',
                'firstSeenDestIp':            'dip',    'firstSeenSrcPort':           'spt',
                'firstSeenDestPort':          'dpt',    'moreFragments':              '_del4_',
                'contFragments':              '_del5_', 'durationSeconds':            'dur',
                'firstSeenSrcPayloadBytes':   '_del6_', 'firstSeenDestPayloadBytes':  '_del7_',
                'firstSeenSrcTotalBytes':     'soct',   'firstSeenDestTotalBytes':    'doct',
                'firstSeenSrcPacketCount':    'spkt',   'firstSeenDestPacketCount':   'dpkt',
                'recordForceOut':             '_del8_'})
df = df.drop([r'^_del\d_$'])
df = df.sample(2_000_000)
xy = rt.xy(df, x_field='soct', y_field='doct', color_by='dpt', align_pixels=False, dot_size='small')
xy._repr_svg_() # force a render to get the xy's to populate
_xvals_, _yvals_ = list(xy.df[xy.x_axis_col]), list(xy.df[xy.y_axis_col])
print(len(_xvals_), len(_yvals_))
_weights_        = [1.0] * len(_xvals_)
#xy

In [ ]:
import random
# num_of_pts   = [100, 200, 50]
num_of_pts   = [40, 80, 25]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [], [], [], []
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])

#
# Simple Renderer
#
def renderSVG(xs, ys, colors, r=0.2):
    x0, y0, x1, y1 = min(xs), min(ys), max(xs), max(ys)
    xperc, yperc   = (x1-x0)*0.01, (y1-y0)*0.01
    x0, y0, x1, y1 = x0-xperc, y0-yperc, x1+xperc, y1+yperc
    svg = []
    svg.append(f'<svg x="0" y="0" width="256" height="256" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">')
    svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" x="0" y="0" fill="#ffffff" />')
    for i in range(len(xs)): svg.append(f'<circle cx="{xs[i]}" cy="{ys[i]}" r="{r}" fill="{colors[i]}" />')
    svg.append('</svg>')
    return ''.join(svg)

#
# xyJustVecs() - why not just use vectors?  Because it doesn't work...
# ... you need the expected density ... which means you need to divide the space into sectors
#
def xyJustVecs(xvals, yvals, weights=None, colors=None, iterations=4, vector_scalar=0.1):
    xvals_last, yvals_last = xvals, yvals
    for iters in range(iterations):
        xvals_next, yvals_next = [], []
        for i in range(len(xvals_last)):
            _x_, _y_ = xvals_last[i], yvals_last[i]
            _u_, _v_ = 0.0, 0.0
            for j in range(len(xvals_last)):
                if i != j:
                    _x_next_, _y_next_ = xvals_last[j], yvals_last[j]
                    _uv_ = rt.unitVector(((_x_, _y_), (_x_next_, _y_next_)))
                    _u_, _v_ = _u_ + _uv_[0], _v_ + _uv_[1]
            _uv_ = rt.unitVector(((0.0, 0.0),(_u_, _v_)))
            _x_next_, _y_next_ = _x_ - _uv_[0]*vector_scalar, _y_ - _uv_[1]*vector_scalar
            xvals_next.append(_x_next_), yvals_next.append(_y_next_)
        xvals_last, yvals_last = xvals_next, yvals_next
    return xvals_last, yvals_last
    
x_new, y_new = xyJustVecs(_xvals_, _yvals_, _weights_, _colors_, iterations=80, vector_scalar=0.1)
rt.tile([renderSVG(_xvals_, _yvals_, _colors_), renderSVG(x_new, y_new, _colors_)], spacer=10)

In [ ]:
#
# xyUniformSampleDistributionSectorTransform() - implementation of the referenced paper
#
def xyUniformSampleDistributionSectorTransformDEBUG(xvals, yvals, weights=None, colors=None, iterations=4, sectors=16, border_perc=0.01, vector_scalar=0.1):
    svgs, svgs_for_sectors = [], []
    # Normalize the coordinates to be between 0.0 and 1.0
    def normalizeCoordinates(xs, ys):
        xmin, ymin, xmax, ymax = min(xs), min(ys), max(xs), max(ys)
        if xmin == xmax: xmin -= 0.0001; xmax += 0.0001
        if ymin == ymax: ymin -= 0.0001; ymax += 0.0001
        xs_new, ys_new = [], []
        for x, y in zip(xs, ys):
            xs_new.append((x-xmin)/(xmax-xmin))
            ys_new.append((y-ymin)/(ymax-ymin))
        return xs_new, ys_new
    # Force all the coordinates to be between 0 and 1
    xvals, yvals = normalizeCoordinates(xvals, yvals)    
    xmin, ymin, xmax, ymax = 0.0, 0.0, 1.0, 1.0
    xperc, yperc = (xmax-xmin)*border_perc, (ymax-ymin)*border_perc
    xmin, ymin, xmax, ymax = xmin-xperc, ymin-yperc, xmax+xperc, ymax+yperc
    # Determine the average density (used for expected density calculations)
    if weights is None: weights = np.ones(len(xvals))
    weight_sum = sum(weights)
    area_total = ((xmax-xmin)*(ymax-ymin))
    density_avg = weight_sum / area_total
    # Determine the side and xy that a specific ray hits
    def sideAndXY(xy, uv):
        _xyi_ = rt.rayIntersectsSegment(xy, uv, (xmin, ymin), (xmax, ymin))
        if _xyi_ is not None: return 0, _xyi_
        _xyi_ = rt.rayIntersectsSegment(xy, uv, (xmax, ymin), (xmax, ymax))
        if _xyi_ is not None: return 1, _xyi_
        _xyi_ = rt.rayIntersectsSegment(xy, uv, (xmax, ymax), (xmin, ymax))
        if _xyi_ is not None: return 2, _xyi_
        _xyi_ = rt.rayIntersectsSegment(xy, uv, (xmin, ymax), (xmin, ymin))
        if _xyi_ is not None: return 3, _xyi_
        # hacking the corner cases ... literally the corners
        if xy[0] >= xmin and xy[0] <= xmax and xy[1] >= ymin and xy[1] <= ymax:
            if uv == (0.0, 0.0):
                print(xy, uv, (xmin,ymin,xmax,ymax))
                raise Exception('No Intersection Found for sideAndXY() ... ray is (0,0)')
            else:
                xp, yp, up, vp = round(xy[0], 2), round(xy[1], 2), round(uv[0], 2), round(uv[1], 2)
                if abs(xp) == abs(yp) and abs(up) == abs(vp):
                    if   up < 0.0 and vp < 0.0: return 0, (xmin, ymin)
                    elif up < 0.0 and vp > 0.0: return 1, (xmax, ymin)
                    elif up > 0.0 and vp > 0.0: return 2, (xmax, ymax)
                    elif up > 0.0 and vp < 0.0: return 3, (xmin, ymax)
                print(xy, uv, (xmin,ymin,xmax,ymax))
                raise Exception('No Intersection Found for sideAndXY() ... xy or uv are not equal to one another')
        else:
            print(xy, uv, (xmin,ymin,xmax,ymax))
            raise Exception('No Intersection Found for sideAndXY() ... point not within bounds')
    # Calculate the sector angles
    _sector_angles_, _sector_anchor_ = [], []
    a, ainc = 0.0, 2*pi/sectors
    for s in range(sectors):
        _sector_angles_.append((a, a+ainc))
        _sector_anchor_.append(a + pi + ainc/2.0)
        a += ainc
    # Calculate the UV vector for a specific point
    def ptUVVec(x,y):
        svg_sectors = [f'<svg x="0" y="0" width="512" height="512" viewBox="{xmin} {ymin} {xmax-xmin} {ymax-ymin}" xmlns="http://www.w3.org/2000/svg">']
        svg_sectors.append(f'<rect x="{xmin}" y="{ymin}" width="{xmax-xmin}" height="{ymax-ymin}" fill="#ffffff" />')
        _sector_sum_ = {}
        for s in range(sectors): _sector_sum_[s] = 0.0
        # Iterate over all points ... adding to the sector sum for the correct sector
        for i in range(len(xvals)):
            _x_, _y_, _w_ = xvals[i], yvals[i], weights[i]
            if _x_ == x and _y_ == y: continue
            _dx_, _dy_ = _x_ - x, _y_ - y
            a = atan2(_dy_, _dx_)
            if a < 0.0: a += 2*pi
            _sector_found_ = False
            for s in range(sectors):
                if a >= _sector_angles_[s][0] and a < _sector_angles_[s][1]:
                    _sector_sum_[s] += _w_
                    _color_ = rt.co_mgr.getColor(s)
                    svg_sectors.append(f'<circle cx="{_x_}" cy="{_y_}" r="0.01" stroke="#000000" stroke-width="0.001" fill="{_color_}" />')
                    svg_sectors.append(f'<line x1="{x}" y1="{y}" x2="{_x_}" y2="{_y_}" stroke="#000000" stroke-width="0.001" />')
                    _sector_found_ = True
                    break
            if not _sector_found_: print('No sector found for point', _x_, _y_, a)
        # Determine the area for each sector (from this points perspective)
        _sector_area_, _poly_definition_ = {}, {}
        for s in range(sectors):
            uv          = (cos(_sector_angles_[s][0]), sin(_sector_angles_[s][0]))
            side_and_xy_a0 = sideAndXY((x,y), uv)
            uv = (cos(_sector_angles_[s][1]), sin(_sector_angles_[s][1]))
            side_and_xy_a1 = sideAndXY((x,y), uv)
            if side_and_xy_a0[0] == side_and_xy_a1[0]: _poly_definition_[s] = [(x,y), side_and_xy_a0[1], side_and_xy_a1[1]] # same side
            else:
                if   side_and_xy_a0[0] == 0 and side_and_xy_a1[0] == 1: _poly_definition_[s] = [(x,y), side_and_xy_a0[1], (xmax,ymin), side_and_xy_a1[1]] # top 
                elif side_and_xy_a0[0] == 1 and side_and_xy_a1[0] == 2: _poly_definition_[s] = [(x,y), side_and_xy_a0[1], (xmax,ymax), side_and_xy_a1[1]] # right
                elif side_and_xy_a0[0] == 2 and side_and_xy_a1[0] == 3: _poly_definition_[s] = [(x,y), side_and_xy_a0[1], (xmin,ymax), side_and_xy_a1[1]] # bottom
                elif side_and_xy_a0[0] == 3 and side_and_xy_a1[0] == 0: _poly_definition_[s] = [(x,y), side_and_xy_a0[1], (xmin,ymin), side_and_xy_a1[1]] # left
            _poly_ = Polygon(_poly_definition_[s])
            _sector_area_[s] = _poly_.area
        # From the paper ... weight the anchor the difference between the expected and actual density
        _scalar_ = vector_scalar
        u, v = 0.0, 0.0
        for s in range(sectors):
            _diff_ = (_sector_sum_[s]/weight_sum) - (_sector_area_[s]/area_total)
            u, v   = u + _scalar_ * _diff_ * cos(_sector_anchor_[s]), v + _scalar_ * _diff_ * sin(_sector_anchor_[s])
            _poly_coords_ = _poly_definition_[s]
            d      = f'M {_poly_coords_[0][0]} {_poly_coords_[0][1]} '
            for i in range(1, len(_poly_coords_)): d += f'L {_poly_coords_[i][0]} {_poly_coords_[i][1]} '
            d += 'Z'
            if _diff_ < 0.0: _color_ = rt.co_mgr.getColor(s) # '#0000ff'
            else:            _color_ = rt.co_mgr.getColor(s) # '#ff0000'
            svg_sectors.append(f'<path d="{d}" stroke="{rt.co_mgr.getColor(s)}" fill="{_color_}" fill-opacity="0.3" stroke-width="0.002"/>')
        # Return the value
        svg_sectors.append(f'<line x1="{x}" y1="{y}" x2="{x+3*u}" y2="{y+3*v}" stroke="#ff0000" stroke-width="0.01" />')
        svg_sectors.append('</svg>')
        svgs_for_sectors.append(''.join(svg_sectors))
        return u,v

    # Iterations...
    for iters in range(iterations):
        svg = [f'<svg x="0" y="0" width="256" height="256" viewBox="{xmin} {ymin} {xmax-xmin} {ymax-ymin}" xmlns="http://www.w3.org/2000/svg">']
        svg.append(f'<rect x="{xmin}" y="{ymin}" width="{xmax-xmin}" height="{ymax-ymin}" x="0" y="0" fill="#ffffff" />')
        xvals_next, yvals_next = [], []
        for j in range(len(xvals)):
            _x_, _y_ = xvals[j], yvals[j]
            uv = ptUVVec(_x_, _y_)
            svg.append(f'<line x1="{_x_}" y1="{_y_}" x2="{_x_+uv[0]}" y2="{_y_+uv[1]}" stroke="#a0a0a0" stroke-width="0.001" />')
            _color_ = colors[j] if colors is not None else '#000000'
            svg.append(f'<circle cx="{_x_}" cy="{_y_}" r="0.004" fill="{_color_}" />')
            _x_next_, _y_next_ = _x_ + uv[0], _y_ + uv[1]
            xvals_next.append(_x_next_), yvals_next.append(_y_next_)
        svg.append('</svg>')
        svgs.append(''.join(svg))
        xvals, yvals = xvals_next, yvals_next
        xvals, yvals = normalizeCoordinates(xvals, yvals)    
        xmin, ymin, xmax, ymax = 0.0, 0.0, 1.0, 1.0
        xperc, yperc = (xmax-xmin)*border_perc, (ymax-ymin)*border_perc
        xmin, ymin, xmax, ymax = xmin-xperc, ymin-yperc, xmax+xperc, ymax+yperc

    # Return
    return xvals, yvals, svgs, svgs_for_sectors

x_new, y_new, svgs, svgs_for_sectors = xyUniformSampleDistributionSectorTransformDEBUG(_xvals_, _yvals_, _weights_, _colors_, iterations=32, border_perc=0.1, vector_scalar=0.1)
rt.table(svgs, per_row=8, spacer=10)

In [ ]:
_to_display_ = []
for i in range(1): _to_display_.append(random.choice(svgs_for_sectors))
rt.table(_to_display_, per_row=4, spacer=10)

In [ ]:
#
# xyUniformSampleDistributionSectorTransform() - implementation of the referenced paper
# ... the above version is debug ... this removes all of the svg creation
#
def xyUniformSampleDistributionSectorTransform(xvals, yvals, weights=None, iterations=4, sectors=16, border_perc=0.01, vector_scalar=0.1):
    # Normalize the coordinates to be between 0.0 and 1.0
    def normalizeCoordinates(xs, ys):
        xmin, ymin, xmax, ymax = min(xs), min(ys), max(xs), max(ys)
        if xmin == xmax: xmin -= 0.0001; xmax += 0.0001
        if ymin == ymax: ymin -= 0.0001; ymax += 0.0001
        xs_new, ys_new = [], []
        for x, y in zip(xs, ys):
            xs_new.append((x-xmin)/(xmax-xmin))
            ys_new.append((y-ymin)/(ymax-ymin))
        return xs_new, ys_new
    # Force all the coordinates to be between 0 and 1
    xvals, yvals = normalizeCoordinates(xvals, yvals)    
    xmin, ymin, xmax, ymax = 0.0, 0.0, 1.0, 1.0
    xperc, yperc = (xmax-xmin)*border_perc, (ymax-ymin)*border_perc
    xmin, ymin, xmax, ymax = xmin-xperc, ymin-yperc, xmax+xperc, ymax+yperc
    # Determine the average density (used for expected density calculations)
    if weights is None: weights = np.ones(len(xvals))
    weight_sum = sum(weights)
    area_total = ((xmax-xmin)*(ymax-ymin))
    density_avg = weight_sum / area_total
    # Determine the side and xy that a specific ray hits
    def sideAndXY(xy, uv):
        _xyi_ = rt.rayIntersectsSegment(xy, uv, (xmin, ymin), (xmax, ymin))
        if _xyi_ is not None: return 0, _xyi_
        _xyi_ = rt.rayIntersectsSegment(xy, uv, (xmax, ymin), (xmax, ymax))
        if _xyi_ is not None: return 1, _xyi_
        _xyi_ = rt.rayIntersectsSegment(xy, uv, (xmax, ymax), (xmin, ymax))
        if _xyi_ is not None: return 2, _xyi_
        _xyi_ = rt.rayIntersectsSegment(xy, uv, (xmin, ymax), (xmin, ymin))
        if _xyi_ is not None: return 3, _xyi_
        # hacking the corner cases ... literally the corners
        if xy[0] >= xmin and xy[0] <= xmax and xy[1] >= ymin and xy[1] <= ymax:
            if uv == (0.0, 0.0):
                print(xy, uv, (xmin,ymin,xmax,ymax))
                raise Exception('No Intersection Found for sideAndXY() ... ray is (0,0)')
            else:
                xp, yp, up, vp = round(xy[0], 2), round(xy[1], 2), round(uv[0], 2), round(uv[1], 2)
                if abs(xp) == abs(yp) and abs(up) == abs(vp):
                    if   up < 0.0 and vp < 0.0: return 0, (xmin, ymin)
                    elif up < 0.0 and vp > 0.0: return 1, (xmax, ymin)
                    elif up > 0.0 and vp > 0.0: return 2, (xmax, ymax)
                    elif up > 0.0 and vp < 0.0: return 3, (xmin, ymax)
                print(xy, uv, (xmin,ymin,xmax,ymax))
                raise Exception('No Intersection Found for sideAndXY() ... xy or uv are not equal to one another')
        else:
            print(xy, uv, (xmin,ymin,xmax,ymax))
            raise Exception('No Intersection Found for sideAndXY() ... point not within bounds')
    # Calculate the sector angles
    _sector_angles_, _sector_anchor_ = [], []
    a, ainc = 0.0, 2*pi/sectors
    for s in range(sectors):
        _sector_angles_.append((a, a+ainc))
        _sector_anchor_.append(a + pi + ainc/2.0)
        a += ainc
    # Calculate the UV vector for a specific point
    def ptUVVec(x,y):
        _sector_sum_ = {}
        for s in range(sectors): _sector_sum_[s] = 0.0
        # Iterate over all points ... adding to the sector sum for the correct sector
        for i in range(len(xvals)):
            _x_, _y_, _w_ = xvals[i], yvals[i], weights[i]
            if _x_ == x and _y_ == y: continue
            _dx_, _dy_ = _x_ - x, _y_ - y
            a = atan2(_dy_, _dx_)
            if a < 0.0: a += 2*pi
            _sector_found_ = False
            for s in range(sectors):
                if a >= _sector_angles_[s][0] and a < _sector_angles_[s][1]:
                    _sector_sum_[s] += _w_
                    _sector_found_   = True
                    break
            if not _sector_found_: print('No sector found for point', _x_, _y_, a)
        # Determine the area for each sector (from this points perspective)
        _sector_area_, _poly_definition_ = {}, {}
        for s in range(sectors):
            uv          = (cos(_sector_angles_[s][0]), sin(_sector_angles_[s][0]))
            side_and_xy_a0 = sideAndXY((x,y), uv)
            uv = (cos(_sector_angles_[s][1]), sin(_sector_angles_[s][1]))
            side_and_xy_a1 = sideAndXY((x,y), uv)
            if side_and_xy_a0[0] == side_and_xy_a1[0]: _poly_definition_[s] = [(x,y), side_and_xy_a0[1], side_and_xy_a1[1]] # same side
            else:
                if   side_and_xy_a0[0] == 0 and side_and_xy_a1[0] == 1: _poly_definition_[s] = [(x,y), side_and_xy_a0[1], (xmax,ymin), side_and_xy_a1[1]] # top 
                elif side_and_xy_a0[0] == 1 and side_and_xy_a1[0] == 2: _poly_definition_[s] = [(x,y), side_and_xy_a0[1], (xmax,ymax), side_and_xy_a1[1]] # right
                elif side_and_xy_a0[0] == 2 and side_and_xy_a1[0] == 3: _poly_definition_[s] = [(x,y), side_and_xy_a0[1], (xmin,ymax), side_and_xy_a1[1]] # bottom
                elif side_and_xy_a0[0] == 3 and side_and_xy_a1[0] == 0: _poly_definition_[s] = [(x,y), side_and_xy_a0[1], (xmin,ymin), side_and_xy_a1[1]] # left
            _poly_ = Polygon(_poly_definition_[s])
            _sector_area_[s] = _poly_.area
        # From the paper ... weight the anchor the difference between the expected and actual density
        _scalar_ = vector_scalar
        u, v = 0.0, 0.0
        for s in range(sectors):
            _diff_ = (_sector_sum_[s]/weight_sum) - (_sector_area_[s]/area_total)
            u, v   = u + _scalar_ * _diff_ * cos(_sector_anchor_[s]), v + _scalar_ * _diff_ * sin(_sector_anchor_[s])
        return u,v

    # Iterations...
    for iters in range(iterations):
        xvals_next, yvals_next = [], []
        for j in range(len(xvals)):
            _x_, _y_ = xvals[j], yvals[j]
            uv = ptUVVec(_x_, _y_)
            _x_next_, _y_next_ = _x_ + uv[0], _y_ + uv[1]
            xvals_next.append(_x_next_), yvals_next.append(_y_next_)
        xvals, yvals = xvals_next, yvals_next
        xvals, yvals = normalizeCoordinates(xvals, yvals)    
        xmin, ymin, xmax, ymax = 0.0, 0.0, 1.0, 1.0
        xperc, yperc = (xmax-xmin)*border_perc, (ymax-ymin)*border_perc
        xmin, ymin, xmax, ymax = xmin-xperc, ymin-yperc, xmax+xperc, ymax+yperc

    # Return
    return xvals, yvals

x_new, y_new = xyUniformSampleDistributionSectorTransform(_xvals_, _yvals_, _weights_, iterations=32, border_perc=0.1, vector_scalar=0.1)

rt.tile([renderSVG(_xvals_, _yvals_, _colors_), renderSVG(x_new, y_new, _colors_, r=0.01)], spacer=10)

In [ ]:
#
# Area of a Triangle (for sector calculation)
# - so, easier if we fix the box to (0.0, 0.0) to (1.0, 1.0)
#   ... could then hard-code the segment intersection function
#
p0, p1, p2 = (0.2, 0.1),(0.9, 0.3),(0.5, 0.9)

# Using Heron's formula (requires sqrt)
a, b, c    = rt.segmentLength((p0,p1)), rt.segmentLength((p1,p2)), rt.segmentLength((p2,p0))
s          = (a+b+c)/2.0
_area_     = sqrt(s*(s-a)*(s-b)*(s-c))

# Using the cross product (least # of multiplies)
_area2_ = abs((p0[0]*(p1[1]-p2[1]) + 
               p1[0]*(p2[1]-p0[1]) + 
               p2[0]*(p0[1]-p1[1]))/2.0)

# Using the shoelace formula
_area3_ = abs((p0[0]*p1[1] - p1[0]*p0[1]) +
              (p1[0]*p2[1] - p2[0]*p1[1]) +
              (p2[0]*p0[1] - p0[0]*p2[1]))/2.0

print(_area_, _area2_, _area3_, Polygon([p0,p1,p2]).area)
Polygon([p0,p1,p2])

In [ ]:
#
# Demonstrates the shoelace formula on the corner sectors
#
_points_ = [(0.6,0.4),(0.0, 0.9),(0.0, 1.0),(0.2, 1.0)]
_sum_    = 0.0
for i in range(len(_points_)):
    p, q   = _points_[i], _points_[(i+1)%len(_points_)]
    _sum_ += p[0]*q[1] - q[0]*p[1] # p_x * q_y - q_x * p_y
_area_ = abs(_sum_)/2.0 # abs required if the order of the points is reversed

print(_area_, Polygon(_points_).area)
Polygon(_points_)

In [ ]:
#
# Demonstrates the shoelace formula on a non-corner sector (if the middle point is just included to make the calculation the same for all sectors)
# ... it may just be simpler to calculate the sector areas outside of the polars use for the sector summation...
#
_points_ = [(0.5,0.3),(0.0, 0.9),(0.0, 0.8),(0.0, 0.6)]
_sum_    = 0.0
for i in range(len(_points_)):
    p, q   = _points_[i], _points_[(i+1)%len(_points_)]
    _sum_ += p[0]*q[1] - q[0]*p[1]
_area_ = abs(_sum_)/2.0 # abs required if the order of the points is reversed

print(_area_, Polygon(_points_).area)
Polygon(_points_)

In [ ]:
#
# Calculate The Area Of Each Sector For Every Point // Prototype
#
p0, p1, p2 = (0.2, 0.1), (0.9, 0.3), (0.5, 0.9) # three points (chosen to be (0,0) -> (1,1))
_lu_       = {'x':[], 'y':[], 'sector':[]}
for _p_ in [p0, p1, p2]:
    for _sector_ in range(16): # 16 sectors per point
        _lu_['x'].append(_p_[0]), _lu_['y'].append(_p_[1]), _lu_['sector'].append(_sector_)

_lu_ = {'x':[], 'y':[], 'sector':[]}
for i in range(10_000):
    _p_ = (0.02 + 0.96*random.random(), 0.02 + 0.96*random.random())
    for _sector_ in range(16): # 16 sectors per point
        _lu_['x'].append(_p_[0]), _lu_['y'].append(_p_[1]), _lu_['sector'].append(_sector_)

df = pl.DataFrame(_lu_)

# Create the sector angle dataframe
_lu_ = {'sector':[], 'a0':[], 'a1':[],              # Sector #, Start and End Angles
        'a0u':[], 'a0v':[], 'a1u':[], 'a1v':[],
        'corner_x':[], 'corner_y':[],               # Corner between segment0 and segment 1
        's0x0':[], 's0x1':[], 's0y0':[], 's0y1':[], # Segment 0
        's1x0':[], 's1x1':[], 's1y0':[], 's1y1':[]} # Segment 1
for _sector_ in range(16):
    _lu_['sector'].append(_sector_)
    _a0_ = _sector_*pi/8.0
    _lu_['a0'].append(_a0_), _lu_['a0u'].append(cos(_a0_)), _lu_['a0v'].append(sin(_a0_))
    _a1_ = (_sector_+1)*pi/8.0
    _lu_['a1'].append(_a1_), _lu_['a1u'].append(cos(_a1_)), _lu_['a1v'].append(sin(_a1_))
    if   _sector_ >= 0 and _sector_ <  4:
        _lu_['s0x0'].append(1.0), _lu_['s0x1'].append(1.0), _lu_['s0y0'].append(0.0), _lu_['s0y1'].append(1.0) # segment 0 (x0,y0) -> (x1,y1) (1,0) -> (1,1)
        _lu_['s1x0'].append(0.0), _lu_['s1x1'].append(1.0), _lu_['s1y0'].append(1.0), _lu_['s1y1'].append(1.0) # segment 1 (x0,y0) -> (x1,y1) (0,1) -> (1,1)
        _lu_['corner_x'].append(1.0), _lu_['corner_y'].append(1.0)
    elif _sector_ >= 4 and _sector_ <  8:
        _lu_['s0x0'].append(0.0), _lu_['s0x1'].append(1.0), _lu_['s0y0'].append(1.0), _lu_['s0y1'].append(1.0) # (0,1) -> (1,1)
        _lu_['s1x0'].append(0.0), _lu_['s1x1'].append(0.0), _lu_['s1y0'].append(0.0), _lu_['s1y1'].append(1.0) # (0,0) -> (0,1)
        _lu_['corner_x'].append(0.0), _lu_['corner_y'].append(1.0)
    elif _sector_ >= 8 and _sector_ < 12:
        _lu_['s0x0'].append(0.0), _lu_['s0x1'].append(0.0), _lu_['s0y0'].append(0.0), _lu_['s0y1'].append(1.0) # (0,0) -> (0,1)
        _lu_['s1x0'].append(0.0), _lu_['s1x1'].append(1.0), _lu_['s1y0'].append(0.0), _lu_['s1y1'].append(0.0) # (0,0) -> (1,0)
        _lu_['corner_x'].append(0.0), _lu_['corner_y'].append(0.0)
    else:
        _lu_['s0x0'].append(0.0), _lu_['s0x1'].append(1.0), _lu_['s0y0'].append(0.0), _lu_['s0y1'].append(0.0) # (0,0) -> (1,0)
        _lu_['s1x0'].append(1.0), _lu_['s1x1'].append(1.0), _lu_['s1y0'].append(0.0), _lu_['s1y1'].append(1.0) # (1,0) -> (1,1)
        _lu_['corner_x'].append(1.0), _lu_['corner_y'].append(0.0)
df_sector_angles = pl.DataFrame(_lu_)
df_sector_angles = df_sector_angles.with_columns((pl.col('s0x1') - pl.col('s0x0')).alias('s0u'), (pl.col('s0y1') - pl.col('s0y0')).alias('s0v'),
                                                 (pl.col('s1x1') - pl.col('s1x0')).alias('s1u'), (pl.col('s1y1') - pl.col('s1y0')).alias('s1v'))

# Join w/ sector information
df = df.join(df_sector_angles, on='sector')

# Create rays for each sector angles
df = df.with_columns((pl.col('a0').cos()).alias('xa0'), (pl.col('a0').sin()).alias('ya0'), # uv for angle 0
                     (pl.col('a1').cos()).alias('xa1'), (pl.col('a1').sin()).alias('ya1')) # uv for angle 1

# Intersect each ray with each segment (uses the multistep version of ray-segment intersection)
# ... determinate "r0s0_det" (ray 0, segment 0) .... done four ways (ray 0 & 1 ... segment 0 & 1)
df = df.with_columns((-pl.col('a0u') * pl.col('s0v') + pl.col('a0v') * pl.col('s0u')).alias('r0s0_det'),
                     (-pl.col('a0u') * pl.col('s1v') + pl.col('a0v') * pl.col('s1u')).alias('r0s1_det'),
                     (-pl.col('a1u') * pl.col('s0v') + pl.col('a1v') * pl.col('s0u')).alias('r1s0_det'),
                     (-pl.col('a1u') * pl.col('s1v') + pl.col('a1v') * pl.col('s1u')).alias('r1s1_det'))
# ... "t" ("r0s0_t") and "u" values ("r0s0_u") (ray 0, segment 0) for all four ways (ray 0 & 1 ... segment 0 & 1)
df = df.with_columns((((pl.col('x') - pl.col('s0x0')) * pl.col('s0v') - (pl.col('y') - pl.col('s0y0')) * pl.col('s0u')) / pl.col('r0s0_det')).alias('r0s0_t'),
                     (((pl.col('x') - pl.col('s0x0')) * pl.col('a0v') - (pl.col('y') - pl.col('s0y0')) * pl.col('a0u')) / pl.col('r0s0_det')).alias('r0s0_u'),

                     (((pl.col('x') - pl.col('s1x0')) * pl.col('s1v') - (pl.col('y') - pl.col('s1y0')) * pl.col('s1u')) / pl.col('r0s1_det')).alias('r0s1_t'),
                     (((pl.col('x') - pl.col('s1x0')) * pl.col('a0v') - (pl.col('y') - pl.col('s1y0')) * pl.col('a0u')) / pl.col('r0s1_det')).alias('r0s1_u'),

                     (((pl.col('x') - pl.col('s0x0')) * pl.col('s0v') - (pl.col('y') - pl.col('s0y0')) * pl.col('s0u')) / pl.col('r1s0_det')).alias('r1s0_t'),
                     (((pl.col('x') - pl.col('s0x0')) * pl.col('a1v') - (pl.col('y') - pl.col('s0y0')) * pl.col('a1u')) / pl.col('r1s0_det')).alias('r1s0_u'),

                     (((pl.col('x') - pl.col('s1x0')) * pl.col('s1v') - (pl.col('y') - pl.col('s1y0')) * pl.col('s1u')) / pl.col('r1s1_det')).alias('r1s1_t'),
                     (((pl.col('x') - pl.col('s1x0')) * pl.col('a1v') - (pl.col('y') - pl.col('s1y0')) * pl.col('a1u')) / pl.col('r1s1_det')).alias('r1s1_u'),)
# ... the x and y intersects (r0s0_xi, r0s0_yi) (ray 0, segment 0) for all four ways (ray 0 & 1) and segment (0 & 1)
df = df.with_columns(pl.when((pl.col('r0s0_t') >= 0.0) & (pl.col('r0s0_u') >= 0.0) & (pl.col('r0s0_u') <= 1.0)).then(pl.col('x') + pl.col('r0s0_t') * pl.col('a0u')).otherwise(None).alias('r0s0_xi'),
                     pl.when((pl.col('r0s0_t') >= 0.0) & (pl.col('r0s0_u') >= 0.0) & (pl.col('r0s0_u') <= 1.0)).then(pl.col('y') + pl.col('r0s0_t') * pl.col('a0v')).otherwise(None).alias('r0s0_yi'),

                     pl.when((pl.col('r0s1_t') >= 0.0) & (pl.col('r0s1_u') >= 0.0) & (pl.col('r0s1_u') <= 1.0)).then(pl.col('x') + pl.col('r0s1_t') * pl.col('a0u')).otherwise(None).alias('r0s1_xi'),
                     pl.when((pl.col('r0s1_t') >= 0.0) & (pl.col('r0s1_u') >= 0.0) & (pl.col('r0s1_u') <= 1.0)).then(pl.col('y') + pl.col('r0s1_t') * pl.col('a0v')).otherwise(None).alias('r0s1_yi'),

                     pl.when((pl.col('r1s0_t') >= 0.0) & (pl.col('r1s0_u') >= 0.0) & (pl.col('r1s0_u') <= 1.0)).then(pl.col('x') + pl.col('r1s0_t') * pl.col('a1u')).otherwise(None).alias('r1s0_xi'),
                     pl.when((pl.col('r1s0_t') >= 0.0) & (pl.col('r1s0_u') >= 0.0) & (pl.col('r1s0_u') <= 1.0)).then(pl.col('y') + pl.col('r1s0_t') * pl.col('a1v')).otherwise(None).alias('r1s0_yi'),

                     pl.when((pl.col('r1s1_t') >= 0.0) & (pl.col('r1s1_u') >= 0.0) & (pl.col('r1s1_u') <= 1.0)).then(pl.col('x') + pl.col('r1s1_t') * pl.col('a1u')).otherwise(None).alias('r1s1_xi'),
                     pl.when((pl.col('r1s1_t') >= 0.0) & (pl.col('r1s1_u') >= 0.0) & (pl.col('r1s1_u') <= 1.0)).then(pl.col('y') + pl.col('r1s1_t') * pl.col('a1v')).otherwise(None).alias('r1s1_yi'),)

df

In [ ]:
# Validation of the above calculations
'''
_tiles_ = []
for k, k_df in df.group_by(['x','y']):
    svg = []
    svg.append('<svg x="0" y="0" width="384" height="384" viewBox="-0.5 -0.5 2.0 2.0" xmlns="http://www.w3.org/2000/svg">')
    svg.append('<rect x="-0.5" y="-0.5" width="2.0" height="2.0" x="0" y="0" fill="#ffffff" />')
    svg.append('<rect x="0" y="0" width="1.0" height="1.0" stroke="#ff0000" stroke-width="0.002" fill="#ffffff" />')
    for i in range(len(k_df)):
        _path_ = f'M {k_df['x'][i]} {k_df['y'][i]}'
        if k_df['r0s0_xi'][i] is not None: _path_ += f' L {k_df["r0s0_xi"][i]} {k_df["r0s0_yi"][i]}'
        if k_df['r0s1_xi'][i] is not None: _path_ += f' L {k_df["r0s1_xi"][i]} {k_df["r0s1_yi"][i]}'
        if k_df['r1s0_xi'][i] is not None: _path_ += f' L {k_df["r1s0_xi"][i]} {k_df["r1s0_yi"][i]}'
        if k_df['r1s1_xi'][i] is not None: _path_ += f' L {k_df["r1s1_xi"][i]} {k_df["r1s1_yi"][i]}'
        _path_ += ' Z'
        _sector_ = k_df['sector'][i]
        svg.append(f'<path d="{_path_}" stroke="#000000" fill="{rt.co_mgr.getColor(_sector_)}" fill-opacity="0.3" stroke-width="0.002"/>') 
    svg.append('</svg>')
    _tiles_.append(''.join(svg))
'''
#rt.table(_tiles_, per_row=8, spacer=10)

In [ ]:
df = df.with_columns(pl.when(pl.col('r0s0_xi').is_not_null()).then(pl.lit('x')).otherwise(pl.lit('_')).alias('r0s0_valid'),
                     pl.when(pl.col('r0s1_xi').is_not_null()).then(pl.lit('x')).otherwise(pl.lit('_')).alias('r0s1_valid'),
                     pl.when(pl.col('r1s0_xi').is_not_null()).then(pl.lit('x')).otherwise(pl.lit('_')).alias('r1s0_valid'),
                     pl.when(pl.col('r1s1_xi').is_not_null()).then(pl.lit('x')).otherwise(pl.lit('_')).alias('r1s1_valid'))
df = df.with_columns(pl.concat_str(['r0s0_valid', 'r0s1_valid', 'r1s0_valid', 'r1s1_valid']).alias('valid'))
# 1M random points... generated the following patterns:
# x_x_ // both rays hit the first segment
# _x_x // both rays hit the second segment
# x__x // first ray hits the first segment, second ray hits the second segment (where the corner occurs)
rt.histogram(df, bin_by='valid', bar_h=26, w=256, h=128)

In [ ]:
df = df.with_columns(pl.when(pl.col('r0s0_xi').is_not_null() & pl.col('r1s0_xi').is_not_null())
                       .then(pl.lit('X_X_').alias('pattern'))
                       .when(pl.col('r0s1_xi').is_not_null() & pl.col('r1s1_xi').is_not_null())
                       .then(pl.lit('_X_X').alias('pattern'))
                       .when(pl.col('r0s0_xi').is_not_null() & pl.col('r1s1_xi').is_not_null())
                       .then(pl.lit('X__X').alias('pattern'))
                       .otherwise(pl.lit('error').alias('pattern')))
rt.histogram(df, bin_by='pattern', bar_h=26, w=256, h=128)

In [ ]:
_points_ = [(0.6,0.4),(0.0, 0.9),(0.0, 1.0),(0.2, 1.0)]
_sum_    = 0.0
for i in range(len(_points_)):
    p, q   = _points_[i], _points_[(i+1)%len(_points_)]
    _sum_ += p[0]*q[1] - q[0]*p[1] # p_x * q_y - q_x * p_y
_area_ = abs(_sum_)/2.0 # abs required if the order of the points is reversed

# Case 0 ... which is X_X_ ... which is the first and second ray both hit the first segment
_c0_0p_x_, _c0_0p_y_, _c0_0q_x_, _c0_0q_y_ = pl.col('r0s0_xi'), pl.col('r0s0_yi'), pl.col('r1s0_xi'), pl.col('r1s0_yi')
_c0_1p_x_, _c0_1p_y_, _c0_1q_x_, _c0_1q_y_ = pl.col('r1s0_xi'), pl.col('r1s0_yi'), pl.col('x'),       pl.col('y')
_c0_2p_x_, _c0_2p_y_, _c0_2q_x_, _c0_2q_y_ = pl.col('x'),       pl.col('y'),       pl.col('r0s0_xi'), pl.col('r0s0_yi')
_c0_op_ = (((_c0_0p_x_*_c0_0q_y_ - _c0_0q_x_*_c0_0p_y_) + (_c0_1p_x_*_c0_1q_y_ - _c0_1q_x_*_c0_1p_y_) + (_c0_2p_x_*_c0_2q_y_ - _c0_2q_x_*_c0_2p_y_))/2.0).abs().alias('area')
_df_ = df.filter(pl.col('pattern') == 'X_X_').with_columns(_c0_op_)
i = 2
_p0_ = _df_[['r0s0_xi', 'r0s0_yi']].to_numpy()[i]
_p1_ = _df_[['r1s0_xi', 'r1s0_yi']].to_numpy()[i]
_p2_ = _df_[['x', 'y']].to_numpy()[i]
_df_['area'][i], Polygon([_p0_, _p1_, _p2_]).area # double checking ... looks really close

In [ ]:
# Case 1 ... which is _X_X ... which is the first and second ray both hit the second segment
_c1_0p_x_, _c1_0p_y_, _c1_0q_x_, _c1_0q_y_ = pl.col('r0s1_xi'), pl.col('r0s1_yi'), pl.col('r1s1_xi'), pl.col('r1s1_yi')
_c1_1p_x_, _c1_1p_y_, _c1_1q_x_, _c1_1q_y_ = pl.col('r1s1_xi'), pl.col('r1s1_yi'), pl.col('x'),       pl.col('y')
_c1_2p_x_, _c1_2p_y_, _c1_2q_x_, _c1_2q_y_ = pl.col('x'),       pl.col('y'),       pl.col('r0s1_xi'), pl.col('r0s1_yi')
_c1_op_ = (((_c1_0p_x_*_c1_0q_y_ - _c1_0q_x_*_c1_0p_y_) + (_c1_1p_x_*_c1_1q_y_ - _c1_1q_x_*_c1_1p_y_) + (_c1_2p_x_*_c1_2q_y_ - _c1_2q_x_*_c1_2p_y_))/2.0).abs().alias('area')
_df_ = df.filter(pl.col('pattern') == '_X_X').with_columns(_c1_op_)
i = 4
_p0_ = _df_[['r0s1_xi', 'r0s1_yi']].to_numpy()[i]
_p1_ = _df_[['r1s1_xi', 'r1s1_yi']].to_numpy()[i]
_p2_ = _df_[['x', 'y']].to_numpy()[i]
_df_['area'][i], Polygon([_p0_, _p1_, _p2_]).area # double checking ... looks really close